In [0]:
# Define the path to your data files
from pyspark import pipelines as pl
from pyspark.sql.functions import *
from pyspark.sql.types import *

# All JSON files inside schema_drift folder in the datastore volume
volume_path = "/Volumes/workspace/damg7370/datastore/schema_drift/customer_data_*.json"


In [0]:
# ---------- BRONZE (ADD NEW COLUMNS) ----------

pl.create_streaming_table("demo_cust_bronze_sd_addcols_p2")

@pl.append_flow(
    target = "demo_cust_bronze_sd_addcols_p2",
    name   = "demo_cust_bronze_sd_addcols_ingest_flow_p2"
)
def demo_cust_bronze_sd_addcols_ingest_flow_p2():
    df = (
        spark.readStream
            .format("cloudFiles")
            .option("cloudFiles.format", "json")
            .option("cloudFiles.inferColumnTypes", "true")
            .option("cloudFiles.schemaEvolutionMode", "addNewColumns")
            .load(volume_path)
    )
    return (
        df.withColumn("ingestion_datetime", current_timestamp())
          .withColumn("source_filename", col("_metadata.file_path"))
    )


In [0]:
# ---------- SILVER (ADD NEW COLUMNS) ----------
@pl.table(
    name="demo_cust_silver_sd_addcols_p2"
)
def demo_cust_silver_sd_addcols_clean_flow_p2():
    df = spark.read.table("demo_cust_bronze_sd_addcols_p2")
    # Add your transformation logic here
    return df